In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from pathlib import Path

# Configure matplotlib
%matplotlib inline

In [ ]:
# Constants
DATA_DIR = Path('../data/raw/subset_50')
FILENAME = "10031606.csv"  # Specify the filename directly

file_path = DATA_DIR / FILENAME

if not file_path.exists():
    print(f"File {FILENAME} not found in {DATA_DIR}. Listing available files:")
    files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.csv')])
    print(files)
    raise FileNotFoundError(f"File {FILENAME} not found.")

print(f"Selected file: {file_path}")

In [ ]:
# Load data
df = pd.read_csv(file_path)
print(f"Loaded {len(df)} rows from {FILENAME}.")

# Create GeoDataFrame for raw GPS points
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
)

# Create GeoDataFrame for matched points
gdf_matched = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.matched_longitude, df.matched_latitude), crs="EPSG:4326"
)

# Convert to Web Mercator (EPSG:3857) for contextily
gdf_wm = gdf.to_crs(epsg=3857)
gdf_matched_wm = gdf_matched.to_crs(epsg=3857)

gdf.head()

In [ ]:
# Plot the trajectory with background map
fig, ax = plt.subplots(figsize=(15, 15))

# Plot raw points
gdf_wm.plot(ax=ax, markersize=10, color='blue', alpha=0.6, label='GPS Points')

# Plot matched points
gdf_matched_wm.plot(ax=ax, markersize=8, color='red', alpha=0.6, label='Matched Points')

# Add background map
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

ax.set_title(f"Trajectory Visualization: {FILENAME}", fontsize=15)
ax.set_axis_off()  # Hide axis for cleaner map view
plt.legend()
plt.show()